In [ ]:
# トークン分類用カスタムモデル
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig

  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    # ボディをロード
    self.roberta = RobertaModel(config, add_pooling_layer=False) # [CLS]トークンによる表現抽出層の無効化
    # トークン分類ヘッドの用意
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    # 重みのロードと初期化
    self.init_weights()

  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
              labels=None, **kwargs):
    # ボディによりエンコーダの表現を取得
    outputs = self.roberta(input_ids, attention_mask=attention_mask,
                           token_type_ids=token_type_ids, **kwargs)
    # 分類器を適用
    sequence_output = self.dropout(outputs[0]) # 最後の隠れ状態
    logits = self.classifier(sequence_output)

    # 損失計算
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    # モデルの出力オブジェクトとして返す
    return TokenClassifierOutput(loss=loss, logits=logits,
                                 hidden_states=outputs.hidden_states,
                                 attentions=outputs.attentions)